In [41]:
import numpy as np
from scipy.linalg import eig
from numpy.linalg import cond

np.random.seed(0)
def print_results(eigenvalues, condition_number):
    print("Eigenvalues:")
    print(eigenvalues)
    print("="*80)
    print("Condition Number: {:.4e}".format(condition_number))

def compute_eigenvalues_and_condition_number(n, R):
    np.random.seed(0)
    # Generate randomly weights and biases and split into two
    b = R * np.random.rand(2 * n)
    w = R * np.random.rand(2 * n)
    b1, b2 = b[:n], b[n:]
    w1, w2 = w[:n], w[n:]
    
    
    print("b:", b)
    L = np.pi  # size of the interval
    delta = L / 8  # overlap
    
    # Initialize matrices
    M1 = np.zeros((n, n))
    M1t = np.zeros((n, n))
    M2 = np.zeros((n, n))
    M2t = np.zeros((n, n))
    M12 = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            # Compute for M1 and M1t
            wij = w1[i] - w1[j]
            vij = w1[i] + w1[j]
            bij = b1[i] - b1[j]
            cij = b1[i] + b1[j]
            if wij != 0:
                M1[i, j] = -(np.sin(0.5 * vij * L + vij * delta + cij) * wij -
                             np.sin(0.5 * wij * L + wij * delta + bij) * vij -
                             np.sin(cij) * wij + np.sin(bij) * vij) / (vij * wij)
                M1t[i, j] = -1 / (delta**2 * wij**3 * vij**3) * (
                    2 * (2 * np.sin(0.5 * wij * L - wij * delta + bij) * delta**2 * vij**3 * wij**2 -
                         2 * np.sin(0.5 * vij * L - vij * delta + cij) * delta**2 * vij**2 * wij**3 -
                         2 * np.cos(0.5 * wij * L - wij * delta + bij) * delta * vij**3 * wij +
                         2 * np.cos(0.5 * vij * L - vij * delta + cij) * delta * vij * wij**3 +
                         vij**3 * np.sin(0.5 * wij * L + wij * delta + bij) -
                         np.sin(0.5 * vij * L + vij * delta + cij) * wij**3 -
                         vij**3 * np.sin(0.5 * wij * L - wij * delta + bij) +
                         np.sin(0.5 * vij * L - vij * delta + cij) * wij**3))
            else:
                M1[i, j] = (np.cos(bij) * w1[i] * L + 2 * np.cos(bij) * w1[i] * delta +
                            np.sin(cij) - np.sin(L * w1[i] + 2 * delta * w1[i] + cij)) / (2 * w1[i])
                M1t[i, j] = 1 / (12 * delta**2 * w1[i]**3) * (
                    32 * np.cos(bij) * delta**3 * w1[i]**3 +
                    24 * delta**2 * np.sin(L * w1[i] - 2 * delta * w1[i] + cij) * w1[i]**2 -
                    12 * np.cos(L * w1[i] - 2 * delta * w1[i] + cij) * delta * w1[i] +
                    3 * np.sin(L * w1[i] + 2 * delta * w1[i] + cij) -
                    3 * np.sin(L * w1[i] - 2 * delta * w1[i] + cij))
            
            # Compute for M2 and M2t
            wij = w2[i] - w2[j]
            vij = w2[i] + w2[j]
            bij = b2[i] - b2[j]
            cij = b2[i] + b2[j]
            if wij != 0:
                M2[i, j] = -(np.sin(0.5 * vij * L + vij * delta + cij) * wij -
                             np.sin(0.5 * wij * L + wij * delta + bij) * vij -
                             np.sin(cij) * wij + np.sin(bij) * vij) / (vij * wij)
                M2t[i, j] = 1 / (delta**2 * wij**3 * vij**3) * (
                    2 * (2 * np.sin(0.5 * wij * L + wij * delta + bij) * delta**2 * vij**3 * wij**2 -
                         2 * np.sin(0.5 * vij * L + vij * delta + cij) * delta**2 * vij**2 * wij**3 +
                         2 * np.cos(0.5 * wij * L + wij * delta + bij) * delta * vij**3 * wij -
                         2 * np.cos(0.5 * vij * L + vij * delta + cij) * delta * vij * wij**3 -
                         vij**3 * np.sin(0.5 * wij * L + wij * delta + bij) +
                         np.sin(0.5 * vij * L + vij * delta + cij) * wij**3 +
                         vij**3 * np.sin(0.5 * wij * L - wij * delta + bij) -
                         np.sin(0.5 * vij * L - vij * delta + cij) * wij**3))
            else:
                M2[i, j] = (np.cos(bij) * w2[i] * L + 2 * np.cos(bij) * w2[i] * delta +
                            np.sin(cij) - np.sin(L * w2[i] + 2 * delta * w2[i] + cij)) / (2 * w2[i])
                M2t[i, j] = -1 / (12 * delta**2 * w2[i]**3) * (
                    -32 * np.cos(bij) * delta**3 * w2[i]**3 +
                    24 * delta**2 * np.sin(L * w2[i] + 2 * delta * w2[i] + cij) * w2[i]**2 +
                    12 * np.cos(L * w2[i] + 2 * delta * w2[i] + cij) * delta * w2[i] -
                    3 * np.sin(L * w2[i] + 2 * delta * w2[i] + cij) +
                    3 * np.sin(L * w2[i] - 2 * delta * w2[i] + cij))
            
            # Compute for M12
            wij = w1[i] - w2[j]
            vij = w1[i] + w2[j]
            bij = b1[i] - b2[j]
            cij = b1[i] + b2[j]
            if wij != 0:
                M12[i, j] = -1 / (delta**2 * wij**3 * vij**3) * (
                    2 * (np.cos(0.5 * wij * L + wij * delta + bij) * delta * vij**3 * wij -
                         np.cos(0.5 * vij * L + vij * delta + cij) * delta * vij * wij**3 +
                         np.cos(0.5 * wij * L - wij * delta + bij) * delta * vij**3 * wij -
                         np.cos(0.5 * vij * L - vij * delta + cij) * delta * vij * wij**3 -
                         vij**3 * np.sin(0.5 * wij * L + wij * delta + bij) +
                         np.sin(0.5 * vij * L + vij * delta + cij) * wij**3 +
                         vij**3 * np.sin(0.5 * wij * L - wij * delta + bij) -
                         np.sin(0.5 * vij * L - vij * delta + cij) * wij**3))
            else:
                M12[i, j] = 1 / (12 * delta**2 * w1[i]**3) * (
                    16 * np.cos(bij) * delta**3 * w1[i]**3 +
                    6 * np.cos(L * w1[i] + 2 * delta * w1[i] + cij) * delta * w1[i] +
                    6 * np.cos(L * w1[i] - 2 * delta * w1[i] + cij) * delta * w1[i] -
                    3 * np.sin(L * w1[i] + 2 * delta * w1[i] + cij) +
                    3 * np.sin(L * w1[i] - 2 * delta * w1[i] + cij))

    # Form the full matrix
    M_full = np.block([[M1 + M1t, M12], [M12.T, M2 + M2t]])
    
    # Compute eigenvalues
    eigenvalues = eig(M_full)[0]
    
    # Compute condition number
    condition_number = cond(M_full)
    
    print_results(eigenvalues, condition_number)
    
    return eigenvalues, condition_number, M1, M1t, M2, M2t, M12, M_full

In [42]:
def condition(matrix):
    condition_number = np.linalg.cond(matrix)
    print("Condition Number: {:.4e}".format(condition_number))
    return 

In [43]:
#Single basis function - Neural Network
def single_neural_net(x,weights,biases,i, sigma): 

    return sigma(weights[i]*x+biases[i])


In [44]:
#Basis function multiplication helper for quadrature
def double_neural_net(x,weights,biases,i,j, sigma): 

    return single_neural_net(x,weights,biases,i,sigma)*single_neural_net(x,weights,biases,j,sigma)

In [45]:
# #Function to build the mass matrix - M
# from scipy.integrate import quad

# def mass_matrix(func,sigma, weights, biases, n):
#     M = np.zeros([n, n])
    
#     for i in range(0, n):
#         for j in range(0, n):
#             M[i, j] = quad(func, 0, 1, args=(weights, biases,i+1,j+1,sigma), limit=1000)[0]

#     return M

In [46]:
n = 5 #half the number of neurons
R = 1 #sample range -R,R
eigenvalues, condition_number, M1, M1t, M2, M2t, M12, M_full = compute_eigenvalues_and_condition_number(n, R)

b: [0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411
 0.43758721 0.891773   0.96366276 0.38344152]
Eigenvalues:
[2.36430835e+01+0.j 1.49835869e+01+0.j 2.14430407e-01+0.j
 3.33888130e-02+0.j 2.85862638e-02+0.j 7.53083402e-03+0.j
 1.50216092e-06+0.j 9.72061690e-07+0.j 1.14866417e-07+0.j
 4.00020866e-08+0.j]
Condition Number: 5.9105e+08


In [47]:
M1

array([[3.07813807, 3.1521907 , 3.05518832, 3.02389928, 1.64122787],
       [3.1521907 , 3.23766996, 3.13585845, 3.09187144, 1.68992668],
       [3.05518832, 3.13585845, 3.0412478 , 2.99201481, 1.63451897],
       [3.02389928, 3.09187144, 2.99201481, 2.98253788, 1.60992658],
       [1.64122787, 1.68992668, 1.63451897, 1.60992658, 0.88438615]])

In [48]:
condition(M1)

Condition Number: 1.3335e+08


In [49]:
condition(M1+M1t)

Condition Number: 1.8465e+08


In [50]:
condition(M1t)

Condition Number: 3.0066e+12


In [51]:
np.linalg.norm(M12,ord=2)

3.5849025544895294

In [52]:
n = 4
R = 2
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

b: [1.09762701 1.43037873 1.20552675 1.08976637 0.8473096  1.29178823
 0.87517442 1.783546  ]
Eigenvalues:
[1.25101966e+01+0.j 7.26889534e+00+0.j 4.41924231e+00+0.j
 3.27128525e+00+0.j 1.06663009e-01+0.j 1.51631672e-02+0.j
 1.59059842e-03+0.j 4.55830353e-04+0.j]
Condition Number: 2.7445e+04


ValueError: too many values to unpack (expected 2)

In [ ]:
n = 4
R = 4
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[12.28223669+0.j  7.89848349+0.j  3.48930954+0.j  4.4315811 +0.j
  1.57181191+0.j  0.09364597+0.j  0.01704627+0.j  0.02516108+0.j]
Condition Number: 7.2052e+02


In [ ]:
n = 4
R = 8
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[6.09601896+0.j 0.54438559+0.j 0.22379801+0.j 3.74647399+0.j
 1.73466476+0.j 2.98117901+0.j 2.6520627 +0.j 2.18486887+0.j]
Condition Number: 2.7239e+01


In [ ]:
n = 4
R = 16
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[7.75251439+0.j 5.44372028+0.j 0.21695148+0.j 0.15795116+0.j
 4.24947659+0.j 2.12558577+0.j 2.35326411+0.j 2.4416427 +0.j]
Condition Number: 4.9082e+01


In [ ]:
n = 4
R = 32
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[6.98804737+0.j 3.9747343 +0.j 3.80555758+0.j 3.22693616+0.j
 2.77309122+0.j 2.1395721 +0.j 2.26863304+0.j 2.51627315+0.j]
Condition Number: 3.2661e+00


In [ ]:
n = 4
R = 64
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[0.41000069+0.j 5.69316853+0.j 5.44516124+0.j 1.20540439+0.j
 2.34402466+0.j 3.00998751+0.j 3.25009673+0.j 3.19277815+0.j]
Condition Number: 1.3886e+01


In [ ]:
n = 4
R = 128
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[3.80429291+0.j 2.28430222+0.j 2.8164818 +0.j 2.90100227+0.j
 3.07534272+0.j 3.04236265+0.j 2.95873785+0.j 2.99550959+0.j]
Condition Number: 1.6654e+00


In [ ]:
n = 4
R = 8
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[8.40460193+0.j 6.224773  +0.j 0.12313323+0.j 0.17962945+0.j
 1.06869016+0.j 3.78241023+0.j 3.55962586+0.j 3.05814846+0.j]
Condition Number: 6.8256e+01


In [ ]:
n = 10
R = 128
eigenvalues, condition_number = compute_eigenvalues_and_condition_number(n, R)

Eigenvalues:
[6.38416303+0.j 5.44282547+0.j 0.11727376+0.j 4.28808689+0.j
 1.57731915+0.j 3.96262209+0.j 1.78033238+0.j 1.98073454+0.j
 2.07974379+0.j 3.72381952+0.j 3.60906129+0.j 2.26056716+0.j
 3.34748239+0.j 3.27859124+0.j 3.29757153+0.j 2.45115268+0.j
 2.89296455+0.j 2.71246993+0.j 2.67460149+0.j 2.69235775+0.j]
Condition Number: 5.4438e+01
